In [171]:
import json
import pandas as pd
from tqdm import tqdm

### Wczytuję klastry nazw

In [40]:
clustered_productsJSON = None
clustered_products = {} # słownik, w którym klucze to nazwy produktów a wartości to numery ich grup
id_to_product = {}      # słownik, w którym klucze to numery grup a wartości to nazwy grup produktów

with open('data/clustered_products.json') as f:
    clustered_productsJSON = json.load(f)  # słownik, którego wartościami są listy zawierające podobne produkty
    
for i, dict_item in enumerate(clustered_productsJSON.items()):
    id_to_product[i] = dict_item[0]
    for prd in dict_item[1]:
        clustered_products[prd] = i  

### Wczytuję dane Shumee

In [19]:
df_org = pd.read_excel('./data/shumee_mckinsey -Aktualizacja 01.03.xlsx', index_col=None, engine='openpyxl',
                      parse_dates=['Data'])

In [150]:
# Dodanie .copy() powoduje, że modyfikowanie df nie wpływa na df_org. 
# Bez tego pandas zgłasza warningi w dalszej części notebooka

df = df_org[['Data', 'Nazwa produktu']].copy(deep=True)
df.head(5)

,Data,Nazwa produktu
0,2021-02-28 23:52:46,PEPCO Ramka na clip 53x63 cm srebrna
1,2021-02-28 23:50:05,shumee Ručně vyrobený kuchyňský dřez se sítkem...
2,2021-02-28 23:50:05,shumee Venkovní solární sprcha se sprchovou hl...
3,2021-02-28 23:48:57,Fontanna ze stali nierdzewnej do basenu ogrodo...
4,2021-02-28 23:39:51,"Krzesła biurowe, 2 szt., pomarańczowe, sztuczn..."


### Uzupełenienie danych Shumee o numer klastra. Zamiana pełnych dat na (rok, miesiąc)

In [151]:
# Numer klastra
df['group'] = df['Nazwa produktu'].apply(lambda x: clustered_products[x] if x in clustered_products else -1)

# Zamiana dat na miesiące
df['ds'] = df['Data'].dt.to_period('M')

df.head(5)

,Data,Nazwa produktu,group,ds
0,2021-02-28 23:52:46,PEPCO Ramka na clip 53x63 cm srebrna,7610,2021-02
1,2021-02-28 23:50:05,shumee Ručně vyrobený kuchyňský dřez se sítkem...,-1,2021-02
2,2021-02-28 23:50:05,shumee Venkovní solární sprcha se sprchovou hl...,-1,2021-02
3,2021-02-28 23:48:57,Fontanna ze stali nierdzewnej do basenu ogrodo...,0,2021-02
4,2021-02-28 23:39:51,"Krzesła biurowe, 2 szt., pomarańczowe, sztuczn...",6619,2021-02


Sumuję, ile razy dana data (rok, miesiąc) pojawiła się w każdej grupie, czyli ile produktów z grupy sprzedano w danym miesiącu

In [152]:
sells_ts = df.groupby('group')['ds'].value_counts().to_frame()
sells_ts.rename(columns = {'ds' : 'Sells'}, inplace=True)
# Teraz sells_ts jest indeksowane multiindeksem (group, ds). Wyciągam group do osobnej kolumny:
sells_ts.reset_index(level=['group'], inplace=True)

sells_ts.head()

,group,Sells
ds,,
2020-11,-1,8745
2020-10,-1,8084
2021-01,-1,4961
2021-02,-1,4497
2020-12,-1,3425


Wygenuję taki dataframe, w którym

* wiersze są indeksowane datami

* nazwy kolumn to nazwy klastrów

W tej chwili każdy wiersz odpowiada miesiącowi w którym coś sprzedano. Chcemy jednak mieć też informacje o miesiącach, kiedy nic nie sprzedano z danej grupy. **months_range** to obiekt klasy PeriodIndex, który zawiera miesiące od pierwszego do ostatniego, o jakich mamy dane.

In [156]:
first_day = df["Data"].dt.date.min()
last_day = df["Data"].dt.date.max()
print(f'months_range zawiera miesiące od {first_day} do {last_day}')
months_range = pd.date_range(start=first_day, end=last_day, freq="M").to_period('M')
months_range

months_range zawiera miesiące od 2018-02-26 do 2021-02-28


PeriodIndex(['2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07',
             '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01',
             '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07',
             '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
             '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
             '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
             '2021-02'],
            dtype='period[M]', freq='M')

In [183]:
dataframes_list = []

for group_id, name in tqdm(id_to_product.items()):
    sells_of_the_cluster = sells_ts[sells_ts['group'] == group_id].copy()
    sells_of_the_cluster.drop('group', axis=1, inplace = True)
    sells_of_the_cluster.rename(columns={'Sells' : name}, inplace = True)
    sells_of_the_cluster = sells_of_the_cluster.reindex(months_range, fill_value = 0)
    dataframes_list.append(sells_of_the_cluster)

100%|██████████| 8739/8739 [00:24<00:00, 362.31it/s]


In [184]:
final_dataframe = pd.concat(dataframes_list, axis=1)

In [186]:
final_dataframe.head(5)

,fontanna do basenu,przypinki,stojak na buty,puzzle,ściana gabionowa,zegar ścienny,dozownik mydła,wklad do szuflady,dywan,pokrywa na basen,...,zestaw szczypiec,zestaw do trejażu,zestaw wieszaków,zjeżdżalnia,zjeżdżalnia dla,zjeżdżalnia xs ślizg,złączka do słupków,bombka z,żółto łańcuch,zwierzak gniotek
2018-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Sprawdźmy, czy wszystko gra:

In [187]:
final_dataframe['pokrywa na basen']

2018-02     0
2018-03     0
2018-04     0
2018-05     0
2018-06     0
2018-07     0
2018-08     0
2018-09     0
2018-10     1
2018-11     0
2018-12     0
2019-01     0
2019-02     0
2019-03     0
2019-04     2
2019-05     2
2019-06    32
2019-07    15
2019-08     3
2019-09     0
2019-10     0
2019-11     0
2019-12     0
2020-01     0
2020-02     2
2020-03     0
2020-04     2
2020-05     2
2020-06     1
2020-07     6
2020-08    14
2020-09     2
2020-10     3
2020-11     0
2020-12     2
2021-01     2
2021-02     5
Freq: M, Name: pokrywa na basen, dtype: int64

In [192]:
final_dataframe.to_csv('./data/sells_time_series.csv')

### Jak wczytywać dane?

Należy przekazać parametr index__col = 0

In [198]:
final_dataframe = pd.read_csv('./data/sells_time_series.csv', index_col=0)

In [200]:
final_dataframe.head()

,fontanna do basenu,przypinki,stojak na buty,puzzle,ściana gabionowa,zegar ścienny,dozownik mydła,wklad do szuflady,dywan,pokrywa na basen,...,zestaw szczypiec,zestaw do trejażu,zestaw wieszaków,zjeżdżalnia,zjeżdżalnia dla,zjeżdżalnia xs ślizg,złączka do słupków,bombka z,żółto łańcuch,zwierzak gniotek
2018-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
